In [2]:
# Import all the required libraries
import pandas as pd
from sentence_transformers import SentenceTransformer,util
from sklearn.metrics.pairwise import cosine_similarity
import subprocess

ModuleNotFoundError: No module named 'pandas'

In [1]:
# Load your CSV data
df = pd.read_csv('/home/pankaj-deshmukh/LocalSend/data_final1.csv')
(df.head())

NameError: name 'pd' is not defined

In [3]:
# Load pre-trained sentence transformer model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Encode the DataFrame rows
df['combined_str'] = df['combined_str'].apply(lambda x: model.encode(x))
# data.head()

def get_similar_items(combined_input, df, model, top_n=5):
    combined_embedding = model.encode(combined_input)
    # Calculate cosine similarity between the combined input and all other combined texts
    similarities = util.pytorch_cos_sim(combined_embedding, df['combined_str'])
    #print(similarities)
    # Get the indices of top N similar items
    similar_indices = similarities.argsort(descending=True, axis=1)[0][:top_n]
    #print(similar_indices)
    # Retrieve the similar items from the DataFrame
    similar_items_df = df.iloc[similar_indices][['age', 'Term', 'Plan', 'Company', 'premium']].reset_index(drop=True)
    # print(similar_items)
    return similar_items_df

In [4]:
def generate_response_with_ollama(prompt):
    # Modify this command to suit your local LLM API or interaction method
    result = subprocess.run(['ollama', 'run', 'mistral'], input=prompt, capture_output=True, text=True) # change it to llama3 for better response but it is slow.
    return result.stdout

In [5]:
# Example query
query = "give me hdfc policies"
response = get_similar_items(query, df, model)
print("Relevant Response Retrieved:")
print(response)

# Combine retrieved data with the query to generate a detailed response
detailed_prompt = f"Based on the data:\n {response.to_dict()},\n Assume your a policy advisor, explain the given policies row wise for the given question: '{query}':"
final_response = generate_response_with_ollama(detailed_prompt)
print(final_response)

/home/pankaj-deshmukh/.local/lib/python3.10/site-packages/sentence_transformers/util.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  a = torch.tensor(a)


<function get_similar_items at 0x762f79d7fbe0>
Relevant Response Retrieved:
   age  Term                   Plan                              Company  \
0   24     7  Click 2 Protect Super  HDFC Life Insurance Company Limited   
1   19     7  Click 2 Protect Super  HDFC Life Insurance Company Limited   
2   18     7  Click 2 Protect Super  HDFC Life Insurance Company Limited   
3   23     7  Click 2 Protect Super  HDFC Life Insurance Company Limited   
4   19     6  Click 2 Protect Super  HDFC Life Insurance Company Limited   

   premium  
0     9898  
1     9605  
2     9605  
3     9730  
4    10178  
